In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Module importing

In [2]:
# Data processing libraries
import pandas as pd
import numpy as np

# NLP libraries
import nltk
from nltk.tag import StanfordPOSTagger

# Machine Learning Libraries
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split # Parameter selection
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

# Other libraries
import time # Execution time of some blocks
import statistics
from IPython.display import display # For displaying DataFrames correctly in Jupyter
from itertools import chain
import collections
import pickle

# Import our own defined functions
from xlm_parsers_functions import *
from drug_interaction_functions import *
from drug_functions import *
from NER_functions import *
from ortographic_features import *
from context_features import *
from feature_creation_interaction import *
from crf_functions import *

/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Objectives of this part
In this second part of the project, we will focus on two different things: 
1. Detection of interactions between drugs
2. Classification of each drug-drug interaction according to one of the following types:
    - Advice: 'Interactions may be expected, and Uroxatral should not be used in combination with other alpha-blockers.'
    - Effect: 'In uninfected volunteers, 46% developed rash while receiving Sustiva and Clarithromycin.'
    - Mechanism: 'Grepafloxacin is a competitive inhibitor of the metabolism of theophylline'.
    - Int: 'The interaction of omeprazole and ketoconazole has been stablished.'

## Reading the XML data

In [3]:
'''
%%time

# Training data

train_data_dir_DrugBank = 'data/Train/DrugBank/'
train_data_dir_MedLine = 'data/Train/MedLine/'
train_data_dirs = [train_data_dir_DrugBank, train_data_dir_MedLine]


XMLdata_train = []
for train_data_dir in train_data_dirs:
    XMLdata_train = XMLdata_train + readXMLData(train_data_dir)
'''

"\n%%time\n\n# Training data\n\ntrain_data_dir_DrugBank = 'data/Train/DrugBank/'\ntrain_data_dir_MedLine = 'data/Train/MedLine/'\ntrain_data_dirs = [train_data_dir_DrugBank, train_data_dir_MedLine]\n\n\nXMLdata_train = []\nfor train_data_dir in train_data_dirs:\n    XMLdata_train = XMLdata_train + readXMLData(train_data_dir)\n"

In [4]:
'''
%%time

# Testing data

test_data_dir_DrugBank = 'data/Test/Test_data_DDI/DrugBank/'
test_data_dir_MedLine = 'data/Test/Test_data_DDI/MedLine/'
test_data_dirs = [test_data_dir_DrugBank, test_data_dir_MedLine]

XMLdata_test = []
for test_data_dir in test_data_dirs:
    XMLdata_test = XMLdata_test + readXMLData(test_data_dir)
'''

"\n%%time\n\n# Testing data\n\ntest_data_dir_DrugBank = 'data/Test/Test_data_DDI/DrugBank/'\ntest_data_dir_MedLine = 'data/Test/Test_data_DDI/MedLine/'\ntest_data_dirs = [test_data_dir_DrugBank, test_data_dir_MedLine]\n\nXMLdata_test = []\nfor test_data_dir in test_data_dirs:\n    XMLdata_test = XMLdata_test + readXMLData(test_data_dir)\n"

## Saving the parsed XML data for later use
As we don't want to read and parse the XML files each time, we will save them into a pickle object that we can quickly read when needed 


In [5]:
'''
with open("parsed_train.txt", "wb") as f:   #Pickling
    pickle.dump(XMLdata_train, f)

with open("parsed_test.txt", "wb") as f:   #Pickling
    pickle.dump(XMLdata_test, f)
'''

'\nwith open("parsed_train.txt", "wb") as f:   #Pickling\n    pickle.dump(XMLdata_train, f)\n\nwith open("parsed_test.txt", "wb") as f:   #Pickling\n    pickle.dump(XMLdata_test, f)\n'

## Read the saved parsed XML data quickly

In [6]:
'''
with open("parsed_train.txt", "rb") as f:   # Unpickling
    XMLdata_train = pickle.load(f)
    
with open("parsed_test.txt", "rb") as f:   # Unpickling
    XMLdata_test = pickle.load(f)

print('Number of training sentences readed: ', len(XMLdata_train))
print('Number of testing sentences readed: ', len(XMLdata_test))
'''

'\nwith open("parsed_train.txt", "rb") as f:   # Unpickling\n    XMLdata_train = pickle.load(f)\n    \nwith open("parsed_test.txt", "rb") as f:   # Unpickling\n    XMLdata_test = pickle.load(f)\n\nprint(\'Number of training sentences readed: \', len(XMLdata_train))\nprint(\'Number of testing sentences readed: \', len(XMLdata_test))\n'

####  Create FrequencyDistribution

We will need it to build a feature

In [7]:
'''
list_of_list_of_tokens=[row[5] for row in XMLdata_train]
list_of_tokens = sum(list_of_list_of_tokens,[])
frequencyDistribution = nltk.FreqDist(list_of_tokens)
'''

'\nlist_of_list_of_tokens=[row[5] for row in XMLdata_train]\nlist_of_tokens = sum(list_of_list_of_tokens,[])\nfrequencyDistribution = nltk.FreqDist(list_of_tokens)\n'

## Creation of features
Before training our model, we need to come up with features to help us determine whether there is a relationship between the two drugs or not.

Some ideas for features are the following:
- Does the sentence contain a modal verb (should, must,...) between the two entities?
- Word bigrams: This is a binary feature for all word bigrams that appeared more than once in the corpus, indicating the presence or absence of each such bigram in the sentence
- Number of words between a pair of drugs
- Number of drugs between a pair of drugs
- POS of words between a pair of drugs: This is a binary feature for word POS tags obtained from POS tagging, and indicates the presence or absence of each POS between the two main drugs.
- Path between a pair of drugs: Path between two main drugs in the parse tree is another feature in our system. Because syntactic paths are in general a sparse feature, we reduced the sparsity by collapsing identical adjacent non-terminal labels. E.g., NP-S-VP-VP-NP is converted to NP-S-VP-NP. This technique decreased the number of paths by 24.8%.

In [8]:
'''
%%time
# Read the database
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()

# Create the train and tests datasets

# Train
X_train = [[text2features(s, drugbank_db,frequencyDistribution)] for s in XMLdata_train]
y_train_int = [[text2interaction(s)] for s in XMLdata_train]
y_train_type = [[text2interactionType(s)] for s in XMLdata_train]

# Test
X_test = [[text2features(s, drugbank_db,frequencyDistribution)] for s in XMLdata_test]
y_test_int = [[text2interaction(s)] for s in XMLdata_test]
y_test_type = [[text2interactionType(s)] for s in XMLdata_test]

print('Number of training sentences: ', len(X_train))
print('Number of testing sentences: ', len(X_test))

'''

"\n%%time\n# Read the database\nwith(open('data/DrugBank_names_DB.txt', 'r')) as f:\n    drugbank_db = f.read().splitlines()\n\n# Create the train and tests datasets\n\n# Train\nX_train = [[text2features(s, drugbank_db,frequencyDistribution)] for s in XMLdata_train]\ny_train_int = [[text2interaction(s)] for s in XMLdata_train]\ny_train_type = [[text2interactionType(s)] for s in XMLdata_train]\n\n# Test\nX_test = [[text2features(s, drugbank_db,frequencyDistribution)] for s in XMLdata_test]\ny_test_int = [[text2interaction(s)] for s in XMLdata_test]\ny_test_type = [[text2interactionType(s)] for s in XMLdata_test]\n\nprint('Number of training sentences: ', len(X_train))\nprint('Number of testing sentences: ', len(X_test))\n\n"

In [9]:
'''
%%time

with open("X_train.txt", "wb") as f:   # Unpickling
    pickle.dump(X_train, f)
    
with open("y_train_int.txt", "wb") as f:   # Unpickling
    pickle.dump(y_train_int, f)
    
with open("y_train_type.txt", "wb") as f:   # Unpickling
    pickle.dump(y_train_type, f)

with open("X_test.txt", "wb") as f:   # Unpickling
    pickle.dump(X_test, f)
    
with open("y_test_int.txt", "wb") as f:   # Unpickling
    pickle.dump(y_test_int, f)
    
with open("y_test_type.txt", "wb") as f:   # Unpickling
    pickle.dump(y_test_type, f)

'''

'\n%%time\n\nwith open("X_train.txt", "wb") as f:   # Unpickling\n    pickle.dump(X_train, f)\n    \nwith open("y_train_int.txt", "wb") as f:   # Unpickling\n    pickle.dump(y_train_int, f)\n    \nwith open("y_train_type.txt", "wb") as f:   # Unpickling\n    pickle.dump(y_train_type, f)\n\nwith open("X_test.txt", "wb") as f:   # Unpickling\n    pickle.dump(X_test, f)\n    \nwith open("y_test_int.txt", "wb") as f:   # Unpickling\n    pickle.dump(y_test_int, f)\n    \nwith open("y_test_type.txt", "wb") as f:   # Unpickling\n    pickle.dump(y_test_type, f)\n\n'

In [10]:

%%time

with open("X_train.txt", "rb") as f:   # Unpickling
    X_train = pickle.load(f)
    
with open("y_train_int.txt", "rb") as f:   # Unpickling
    y_train_int = pickle.load(f)
    
with open("y_train_type.txt", "rb") as f:   # Unpickling
    y_train_type = pickle.load(f)

with open("X_test.txt", "rb") as f:   # Unpickling
    X_test = pickle.load(f)
    
with open("y_test_int.txt", "rb") as f:   # Unpickling
    y_test_int = pickle.load(f)
    
with open("y_test_type.txt", "rb") as f:   # Unpickling
    y_test_type = pickle.load(f)

print('Number of training sentences readed: ', len(X_train))
print('Number of testing sentences readed: ', len(X_test))
''''''

Number of training sentences readed:  27792
Number of testing sentences readed:  5716
CPU times: user 2.08 s, sys: 351 ms, total: 2.43 s
Wall time: 2.49 s


# Model training

## First model: Classifying interactions between true/false

## Model training
Using training data and the parameters obtained in the previous step

In [11]:
%%time

mod1 = trainCRFAndEvaluate(
            X_train = X_train, 
            y_train = y_train_int,
            X_test = X_test,
            y_test = y_test_int,
            labels = ['true', 'false'],
            hyperparam_optim = True)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 37.1min finished


Hyperparameter optimization took 2257.54 seconds to complete
best params: {'c1': 1.0458554701644089, 'c2': 0.006237772066168241}
best CV score: 0.867758006638595
model size: 0.66M
             precision    recall  f1-score   support

       true      0.524     0.531     0.527       979
      false      0.903     0.900     0.901      4737

avg / total      0.838     0.837     0.837      5716

CPU times: user 10min 15s, sys: 54.1 s, total: 11min 9s
Wall time: 38min 14s


In [12]:
y_pred_int = mod1.predict(X_test)

## Second model: Classifying between types of interactions

### Filtering the data

In [13]:
X_train_filtered = []
y_train_filtered = []
for idx,val in enumerate(y_train_int):
    if val==['true']:
        X_train_filtered.append(X_train[idx])
        y_train_filtered.append(y_train_type[idx]) # no we don't want true or false rather than int, mechanism...


X_test_filtered = []
y_test_filtered = []
for idx,val in enumerate(y_pred_int):
    if val==['true']:
        X_test_filtered.append(X_test[idx])
        y_test_filtered.append(y_test_type[idx]) # no we don't want true or false rather than int, mechanism...

print('Number of training sentences: ', len(X_train_filtered))
print('Number of testing sentences: ', len(X_test_filtered))

Number of training sentences:  4021
Number of testing sentences:  993


In [14]:
mod2 = trainCRFAndEvaluate(
            X_train = X_train_filtered, 
            y_train = y_train_filtered,
            X_test = X_test_filtered,
            y_test = y_test_filtered,
            labels = ['mechanism', 'advise', 'effect', 'int'],
            hyperparam_optim = True)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_f

Hyperparameter optimization took 384.2 seconds to complete
best params: {'c1': 0.017187836718689203, 'c2': 0.0056279023357100012}
best CV score: 0.7362345683497273
model size: 0.88M
             precision    recall  f1-score   support

  mechanism      0.332     0.874     0.482       135
     advise      0.421     0.726     0.533       124
     effect      0.423     0.866     0.569       194
        int      0.556     0.224     0.319        67

avg / total      0.416     0.752     0.505       520



In [15]:
y_pred_type = mod2.predict(X_test_filtered)

# Join the obtained predictions with the predictions made in the first model
joinResultsFirstSecondModel(y_test_type = y_test_type, y_pred_type = y_pred_type, y_pred_int = y_pred_int)

             precision    recall  f1-score   support

  mechanism      0.332     0.391     0.359       302
     advise      0.421     0.407     0.414       221
     effect      0.423     0.467     0.444       360
        int      0.556     0.156     0.244        96

avg / total      0.408     0.399     0.391       979



In [16]:
'''

%%time
# Define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

# Parameter search

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=1,
                        scoring=f1_scorer)

rs.fit(X_train, y_int_train)
'''

"\n\n%%time\n# Define fixed parameters and parameters to search\ncrf = sklearn_crfsuite.CRF(\n    algorithm='lbfgs',\n    max_iterations=100,\n    all_possible_transitions=True\n)\n\n# Parameter search\n\n# use the same metric for evaluation\nf1_scorer = make_scorer(metrics.flat_f1_score,\n                        average='weighted')\n\nparams_space = {\n    'c1': scipy.stats.expon(scale=0.5),\n    'c2': scipy.stats.expon(scale=0.05),\n}\n\nrs = RandomizedSearchCV(crf, params_space,\n                        cv=3,\n                        verbose=1,\n                        n_jobs=-1,\n                        n_iter=1,\n                        scoring=f1_scorer)\n\nrs.fit(X_train, y_int_train)\n"

In [17]:
'''
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))
'''



"\n# crf = rs.best_estimator_\nprint('best params:', rs.best_params_)\nprint('best CV score:', rs.best_score_)\nprint('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))\n"

In [18]:
'''
_x = [s.parameters['c1'] for s in rs.grid_scores_]
_y = [s.parameters['c2'] for s in rs.grid_scores_]
_c = [s.mean_validation_score for s in rs.grid_scores_]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))
'''

'\n_x = [s.parameters[\'c1\'] for s in rs.grid_scores_]\n_y = [s.parameters[\'c2\'] for s in rs.grid_scores_]\n_c = [s.mean_validation_score for s in rs.grid_scores_]\n\nfig = plt.figure()\nfig.set_size_inches(12, 12)\nax = plt.gca()\nax.set_yscale(\'log\')\nax.set_xscale(\'log\')\nax.set_xlabel(\'C1\')\nax.set_ylabel(\'C2\')\nax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(\n    min(_c), max(_c)\n))\n\nax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])\n\nprint("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))\n'

In [19]:
'''
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(collections.Counter(crf.state_features_).most_common(15))

print("\nTop negative:")
print_state_features(collections.Counter(crf.state_features_).most_common()[-15:])
'''

'\ndef print_state_features(state_features):\n    for (attr, label), weight in state_features:\n        print("%0.6f %-8s %s" % (weight, label, attr))\n\nprint("Top positive:")\nprint_state_features(collections.Counter(crf.state_features_).most_common(15))\n\nprint("\nTop negative:")\nprint_state_features(collections.Counter(crf.state_features_).most_common()[-15:])\n'